# 台指選擇權put_call_ratio值抓取  
* 透過openapi取得  
> [臺指選擇權Put/Call比](https://openapi.taifex.com.tw/#/%E8%B3%87%E6%96%99%E6%9F%A5%E8%A9%A2API/get_PutCallRatio)
* 舊數據透過網頁查詢下載csv檔案, 並將csv檔案轉存至duckdb  
> 台灣期交所首頁 > 交易資訊 > 交易資訊 選擇權 > 臺指選擇權Put/Call比  
> [臺指選擇權(TXO)Put/Call Ratios](https://www.taifex.com.tw/cht/3/pcRatio)

In [1]:
import os
from datetime import date
from datetime import datetime
import shutil
import requests
import pandas as pd
import duckdb
import zipfile

In [2]:
# 引用自建公用模組
from proj_util_pkg.settings import ProjEnvSettings

## 公用參數設定

In [3]:
# 欄數全展開
pd.set_option("display.max_columns", None)

In [4]:
# 新增偽裝成chrome瀏覽器的標頭
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

## 外部資料讀取  
### 方法1: 透過期交所 open api 取得最近一個月數據  
* API無法取得當天最新數據，只能取得Ｄ-1交易日資訊

In [5]:
# 取得散戶小台淨未平倉口數資料
taifex_put_call_ratio = requests.get(
    "https://openapi.taifex.com.tw/v1/PutCallRatio", 
    verify=False,
    headers=headers
).json()
print(taifex_put_call_ratio)

[{'Date': '20241001', 'PutVolume': '333926', 'CallVolume': '362979', 'PutCallVolumeRatio%': '92.00', 'PutOI': '162916', 'CallOI': '205574', 'PutCallOIRatio%': '79.25'}, {'Date': '20240930', 'PutVolume': '406677', 'CallVolume': '459360', 'PutCallVolumeRatio%': '88.53', 'PutOI': '145148', 'CallOI': '185786', 'PutCallOIRatio%': '78.13'}, {'Date': '20240927', 'PutVolume': '310160', 'CallVolume': '364186', 'PutCallVolumeRatio%': '85.17', 'PutOI': '145819', 'CallOI': '147529', 'PutCallOIRatio%': '98.84'}, {'Date': '20240926', 'PutVolume': '210600', 'CallVolume': '207330', 'PutCallVolumeRatio%': '101.58', 'PutOI': '127002', 'CallOI': '116881', 'PutCallOIRatio%': '108.66'}, {'Date': '20240925', 'PutVolume': '599264', 'CallVolume': '590677', 'PutCallVolumeRatio%': '101.45', 'PutOI': '91441', 'CallOI': '75186', 'PutCallOIRatio%': '121.62'}, {'Date': '20240924', 'PutVolume': '338022', 'CallVolume': '348765', 'PutCallVolumeRatio%': '96.92', 'PutOI': '183324', 'CallOI': '139455', 'PutCallOIRatio%':

/Users/bryson0083/projects/bryson_stock/.venv/lib/python3.12/site-packages/urllib3/connectionpool.py:1061: InsecureRequestWarning: Unverified HTTPS request is being made to host 'openapi.taifex.com.tw'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [6]:
# json 轉成 dataframe
taifex_put_call_ratio_df = pd.DataFrame(taifex_put_call_ratio)
print(taifex_put_call_ratio_df.shape)
taifex_put_call_ratio_df

(19, 7)


,Date,PutVolume,CallVolume,PutCallVolumeRatio%,PutOI,CallOI,PutCallOIRatio%
0,20241001,333926,362979,92.00,162916,205574,79.25
1,20240930,406677,459360,88.53,145148,185786,78.13
2,20240927,310160,364186,85.17,145819,147529,98.84
3,20240926,210600,207330,101.58,127002,116881,108.66
4,20240925,599264,590677,101.45,91441,75186,121.62
5,20240924,338022,348765,96.92,183324,139455,131.46
6,20240923,238460,221153,107.83,159297,133627,119.21
7,20240920,272741,261526,104.29,142159,114246,124.43
8,20240919,240876,211959,113.64,112669,90982,123.84
9,20240918,473574,520175,91.04,57842,58371,99.09


### 方法2: 透過期交所網站下載資料進行更新  
* 這方法可以取得最新交易日當天的資訊  

In [7]:
# 設置請求頭,模擬瀏覽器行為
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
    'Accept-Language': 'zh-TW,zh;q=0.9,en-US;q=0.8,en;q=0.7',
    'Accept-Encoding': 'gzip, deflate, br',
    'Connection': 'keep-alive',
    'Upgrade-Insecure-Requests': '1',
    'Referer': 'https://www.taifex.com.tw/cht/3/pcRatio'
}

# 日期參數設定
# input_date = "2024/08/05"  # 保留作為手動指定日期
input_date = date.today().strftime("%Y/%m/%d")  # 預設指定當天日期

end_date = pd.to_datetime(input_date)
last_month = (end_date - pd.DateOffset(days=30)).strftime("%Y/%m/%d")

# 設置請求參數
# payload = {
#     'down_type': '',
#     'queryStartDate': '2024/09/04',
#     'queryEndDate': '2024/10/04'
# }
payload = {
    'down_type': '',
    'queryStartDate': last_month,
    'queryEndDate': end_date.strftime("%Y/%m/%d")
}
print(payload)

# 發送POST請求
url = 'https://www.taifex.com.tw/cht/3/pcRatioDown'
response = requests.post(url, headers=headers, data=payload)

# 檢查請求是否成功
if response.status_code == 200:
    # 將內容保存為CSV檔案
    with open('tw_pc_ratio.csv', 'wb') as file:
        file.write(response.content)
    print("檔案已成功下載為 tw_pc_ratio.csv")

    # 讀取CSV檔案並顯示內容，設定no header
    tw_pc_ratio_df = pd.read_csv('tw_pc_ratio.csv', skiprows=1, header=None, encoding='big5')
    
    # tw_pc_ratio_df移除最後一欄都是空的資料
    tw_pc_ratio_df = tw_pc_ratio_df.dropna(how='all', axis=1)
    tw_pc_ratio_df.columns = taifex_put_call_ratio_df.columns
    tw_pc_ratio_df.loc[:,"Date"] = pd.to_datetime(tw_pc_ratio_df["Date"])

    os.remove('tw_pc_ratio.csv')
else:
    print(f"請求失敗,狀態碼: {response.status_code}")


{'down_type': '', 'queryStartDate': '2024/09/05', 'queryEndDate': '2024/10/05'}
檔案已成功下載為 tw_pc_ratio.csv


In [8]:
# 將tw_pc_ratio_df 複製給 taifex_put_call_ratio_df
taifex_put_call_ratio_df = tw_pc_ratio_df.copy()
taifex_put_call_ratio_df.head(3)

,Date,PutVolume,CallVolume,PutCallVolumeRatio%,PutOI,CallOI,PutCallOIRatio%
0,2024-10-04 00:00:00,605684,621014,97.53,106443,102017,104.34
1,2024-10-01 00:00:00,333926,362979,92.00,162916,205574,79.25
2,2024-09-30 00:00:00,406677,459360,88.53,145148,185786,78.13


## 資料留存ＤＢ

In [9]:
# 設定資料庫路徑
TWSTOCK_DATA_ROOT = os.environ.get("hist_data_path")
twstock_db_path = f"{TWSTOCK_DATA_ROOT}/twstock.duckdb"

In [10]:
# 連線資料庫
conn_duckdb = duckdb.connect(twstock_db_path)

In [11]:
table_name = "tw_pc_ratio"

In [12]:
# 針對DataFrame，以Date單筆先確認表中，欄位Date沒有重複資料時，才進行單筆insert
for i in range(len(taifex_put_call_ratio_df)):
    try:
        taifex_put_call_ratio_df.iloc[i:i+1].to_sql(table_name, conn_duckdb, if_exists="append", index=False)
    except Exception as e:
        # print(e)
        pass

/var/folders/14/b8c6vz6n79n4wt12qny6djk40000gn/T/ipykernel_48857/14700357.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  taifex_put_call_ratio_df.iloc[i:i+1].to_sql(table_name, conn_duckdb, if_exists="append", index=False)


In [13]:
# 查詢表中所有資料
conn_duckdb.execute(f"SELECT * FROM {table_name} order by Date desc").fetch_df()

,Date,PutVolume,CallVolume,PutCallVolumeRatio%,PutOI,CallOI,PutCallOIRatio%
0,2024-10-04,605684,621014,97.53,106443,102017,104.34
1,2024-10-01,333926,362979,92.00,162916,205574,79.25
2,2024-09-30,406677,459360,88.53,145148,185786,78.13
3,2024-09-27,310160,364186,85.17,145819,147529,98.84
4,2024-09-26,210600,207330,101.58,127002,116881,108.66
5,2024-09-25,599264,590677,101.45,91441,75186,121.62
6,2024-09-24,338022,348765,96.92,183324,139455,131.46
7,2024-09-23,238460,221153,107.83,159297,133627,119.21
8,2024-09-20,272741,261526,104.29,142159,114246,124.43
9,2024-09-19,240876,211959,113.64,112669,90982,123.84


In [14]:
# 關閉資料庫連線
conn_duckdb.close()